<a href="https://colab.research.google.com/github/ravikumarn24/Glaucoma-Detection/blob/main/GlaucomaDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth,files
from oauth2client.client import GoogleCredentials
import keras
import os
import numpy as np
from numpy.random import seed
seed(1234)
import tensorflow as tf
tf.set_random_seed(1234)
import cv2
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input, Dropout, merge, concatenate, subtract, multiply, \
    BatchNormalization, Activation, AveragePooling2D
from keras.models import Sequential, Model, load_model
import csv
from google.colab import files
#files.upload()

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print(os.listdir(os.getcwd()))
#files.upload()
# Create & upload a file.
fid = drive.ListFile({'q':"title='siamese_cnn_for_glaucoma_model.h5'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('siamese_cnn_for_glaucoma_model.h5')
fid = drive.ListFile({'q':"title='glaucoma.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('glaucoma.zip')
!unzip glaucoma.zip

    100% |████████████████████████████████| 993kB 19.6MB/s 


Using TensorFlow backend.


['.config', 'adc.json', 'sample_data']
Archive:  glaucoma.zip
  inflating: glaucoma/ground_truth.csv  
   creating: glaucoma/ref/
  inflating: glaucoma/ref/01_g.jpg   
  inflating: glaucoma/ref/02_g.jpg   
  inflating: glaucoma/ref/03_g.jpg   
  inflating: glaucoma/ref/04_g.jpg   
  inflating: glaucoma/ref/05_g.jpg   
  inflating: glaucoma/ref/06_g.jpg   
  inflating: glaucoma/ref/07_g.jpg   
  inflating: glaucoma/ref/08_g.jpg   
  inflating: glaucoma/ref/09_g.jpg   
  inflating: glaucoma/ref/10_g.jpg   
  inflating: glaucoma/ref/11_g.jpg   
  inflating: glaucoma/ref/12_g.jpg   
  inflating: glaucoma/ref/13_g.jpg   
  inflating: glaucoma/ref/14_g.jpg   
  inflating: glaucoma/ref/15_g.jpg   
   creating: glaucoma/reference/
  inflating: glaucoma/reference/01_g.jpg  
  inflating: glaucoma/reference/02_g.jpg  
  inflating: glaucoma/reference/03_g.jpg  
  inflating: glaucoma/reference/04_g.jpg  
  inflating: glaucoma/reference/05_g.jpg  
  inflating: glaucoma/reference/06_g.jpg  
  inflati

In [ ]:

dataset_folder = r"glaucoma/"
ground_truth_file = dataset_folder+r"ground_truth.csv"

reference_folder = dataset_folder + r"ref"
train_folder = dataset_folder + r"tr"
test_folder = dataset_folder + r"ts"


train_csv = csv.reader(open(ground_truth_file,"r"))
train_label = [row for row in train_csv]

train_label = train_label[4:]
reference_imgs = []
normal_imgs = []
glaucoma_imgs = []
img_list = []
print("reference image generation ")
for name in os.listdir(reference_folder):
    path = reference_folder + "/" + name
    imgref = cv2.imread(path)
    imgref = cv2.resize(imgref,(512, 512))
    img_list = [imgref] #cv2.flip(imgref, flipCode=0), cv2.flip(imgref, flipCode=1), cv2.flip(imgref, flipCode=-1)]
    for img in img_list:
        reference_imgs.append(img[ : , : , : ].reshape([img.shape[0],img.shape[1],3]))
    img_list.clear()

print("train image generation")
for name in os.listdir(train_folder):
    print(name)
    path = train_folder + "/" + name
    imginp = cv2.imread(path)
    imginp = cv2.resize(imginp,(512, 512))
    img_list = [imginp]
    subname = name.split(".")[0]
    #print(subname)
    for row in train_label:
        img_name = row[0][:-1]
        if img_name == subname :
            if row[-2] == "Normal":
                for img in img_list:
                    normal_imgs.append(img[:, :, :].reshape([img.shape[0], img.shape[1], 3]))
            else:
                for img in img_list:
                    glaucoma_imgs.append(img[:, :, :].reshape([img.shape[0], img.shape[1], 3]))
            break
    img_list.clear()

for name in os.listdir(test_folder):
    path = test_folder + "/" + name
    imginp = cv2.imread(path)
    imginp = cv2.resize(imginp,(512, 512))
    img_list = [imginp]
    subname = name.split(".")[0]
    #print(subname)
    for row in train_label:
        img_name = row[0][:-1]
        if img_name == subname :
            if row[-2] == "Normal":
                for img in img_list:
                    normal_imgs.append(img[:, :, :].reshape([img.shape[0], img.shape[1], 3]))
            else:
                for img in img_list:
                    glaucoma_imgs.append(img[:, :, :].reshape([img.shape[0], img.shape[1], 3]))
            break
    img_list.clear()

normal_imgs = np.array(normal_imgs)
np.random.shuffle(normal_imgs)

glaucoma_imgs = np.array(glaucoma_imgs)
np.random.shuffle(glaucoma_imgs)

glaucoma_split = int(0.6 * len(glaucoma_imgs))
normal_split = int(0.6 * len(normal_imgs))

test_glaucoma_imgs = glaucoma_imgs[glaucoma_split : ]
test_normal_imgs = normal_imgs[normal_split : ]

glaucoma_imgs = glaucoma_imgs[ : glaucoma_split]
normal_imgs = normal_imgs [ : normal_split]

similar_pairs = []
dissimilar_pairs = []
for imgref in reference_imgs:
    for imginp in glaucoma_imgs:
        similar_pairs.append((imgref, imginp))
    for imginp in normal_imgs:
        dissimilar_pairs.append((imgref, imginp))

for i in range(len(glaucoma_imgs)):
    for j in range(i+1,len(glaucoma_imgs),1):
        similar_pairs.append((glaucoma_imgs[i],glaucoma_imgs[j]))

for i in range(len(reference_imgs)):
    for j in range(i+1,len(reference_imgs),1):
        similar_pairs.append((reference_imgs[i],reference_imgs[j]))

for imginp1 in glaucoma_imgs:
    for imginp2 in normal_imgs:
        dissimilar_pairs.append((imginp1,imginp2))

# print(len(similar_pairs))
# print(len(dissimilar_pairs))

similar_gt = np.ones(len(similar_pairs))
dissimilar_gt = np.zeros(len(dissimilar_pairs))

total_pairs = similar_pairs + dissimilar_pairs
total_pairs = np.array(total_pairs)
# print(len(total_pairs))


ground_truth = np.concatenate([similar_gt, dissimilar_gt])
# print(len(ground_truth))

idxs = np.arange(len(total_pairs))
np.random.shuffle(idxs)

total_pairs = total_pairs[idxs]
ground_truth = ground_truth[idxs]

reference image generation 
train image generation
drishtiGS_026.png
drishtiGS_016.png
drishtiGS_004.png
drishtiGS_035.png
drishtiGS_075.png
drishtiGS_101.png
drishtiGS_058.png
drishtiGS_084.png
drishtiGS_040.png
drishtiGS_018.png
drishtiGS_063.png
drishtiGS_081.png
drishtiGS_012.png
drishtiGS_051.png
drishtiGS_057.png
drishtiGS_046.png
drishtiGS_098.png
drishtiGS_068.png
drishtiGS_042.png
drishtiGS_032.png
drishtiGS_094.png
drishtiGS_038.png
drishtiGS_080.png
drishtiGS_062.png
drishtiGS_061.png
drishtiGS_033.png
drishtiGS_066.png
drishtiGS_044.png
drishtiGS_076.png
drishtiGS_045.png
drishtiGS_031.png
drishtiGS_049.png
drishtiGS_015.png
drishtiGS_002.png
drishtiGS_036.png
drishtiGS_047.png
drishtiGS_024.png
drishtiGS_092.png
drishtiGS_010.png
drishtiGS_064.png
drishtiGS_088.png
drishtiGS_060.png
drishtiGS_069.png
drishtiGS_037.png
drishtiGS_041.png
drishtiGS_090.png
drishtiGS_008.png
drishtiGS_017.png
drishtiGS_089.png
drishtiGS_022.png


In [ ]:
####-----MODEL CONFIGURATION-----####
input_shape = (512, 512, 3)

Input1 = Input(shape=input_shape)
Input2 = Input(shape=input_shape)

convnet = Sequential()

# module
#convnet.add(Conv2D(64, (7, 7), activation='relu', input_shape=input_shape))
convnet.add(Conv2D(64, (7, 7), activation='relu'))
convnet.add(BatchNormalization())
convnet.add(MaxPooling2D((2, 2)))
convnet.add(Conv2D(64, (3, 3), activation='relu'))
convnet.add(Conv2D(64, (3, 3), activation='relu'))
convnet.add(BatchNormalization())
convnet.add(MaxPooling2D((2, 2)))
convnet.add(Conv2D(64, (3, 3), activation='relu'))
convnet.add(Conv2D(64, (3, 3), activation='relu'))
convnet.add(BatchNormalization())
convnet.add(MaxPooling2D((2, 2)))
convnet.add(Conv2D(128, (3, 3), activation='relu'))
convnet.add(Conv2D(128, (3, 3), activation='relu'))
convnet.add(BatchNormalization())
convnet.add(MaxPooling2D((2, 2)))
convnet.add(Conv2D(128, (3, 3), activation='relu'))
convnet.add(Conv2D(128, (3, 3), activation='relu'))
convnet.add(BatchNormalization())
convnet.add(MaxPooling2D((2, 2)))
convnet.add(Conv2D(256, (3, 3), activation='relu'))
convnet.add(Conv2D(256, (3, 3), activation='relu'))
convnet.add(BatchNormalization())
convnet.add(MaxPooling2D((2, 2)))
convnet.add(Conv2D(512, (3, 3), activation='relu'))
#convnet.add(Conv2D(256, (3,3), activation='relu'))
convnet.add(BatchNormalization())
#convnet.add(MaxPooling2D((2,2)))
#convnet.add(Conv2D(512, (3, 3), activation='relu'))
#convnet.add(Conv2D(512, (3,3), activation='relu'))
convnet.add(Flatten())
#convnet.summary()
#convnet = Model(input=input_layer, output=flatten)

final_feature1 = convnet(Input1)
final_feature2 = convnet(Input2)

#convnet = ResNet50(weights='imagenet', include_top =False)
#convnet.trainable = False

#feature1 = convnet(Input1)
#final_feature1 = Flatten()(feature1)
#feature2 = convnet(Input2)
#final_feature2 = Flatten()(feature2)

subtract_feature = subtract([final_feature1, final_feature2])
merged_feature = multiply([subtract_feature, subtract_feature])

pre_final_layer = Dense(512, activation='sigmoid')(merged_feature)
dropout_layer = Dropout(0.5)(pre_final_layer)
final_layer = Dense(64, activation='sigmoid')(dropout_layer)
final_dropout_layer = Dropout(0.5)(final_layer)
prediction = Dense(1, activation='sigmoid')(final_dropout_layer)

siamese_net = Model(input=[Input1, Input2], output=prediction)

siamese_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc"])

convnet.summary()
siamese_net.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 506, 506, 64)      9472      
_________________________________________________________________
batch_normalization_1 (Batch (None, 506, 506, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 253, 253, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 251, 251, 64)      36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 249, 249, 64)      36928     
_________________________________________________________________
batch_normalization_2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [ ]:
#siamese_net.fit([total_pairs[:, 0], total_pairs[:, 1]], ground_truth, batch_size=12, epochs=7  ,validation_split=0.1)

siamese_net = load_model('siamese_cnn_for_glaucoma_model.h5')

#siamese_net.save("siamese_for_glaucoma.h5")

#siamese_net = load_model("siamese_for_glaucoma.h5")
test_imgs = []
test_ground_truth = []

for img in test_glaucoma_imgs:
    test_imgs.append(img)
    test_ground_truth.append(1)

for img in test_normal_imgs:
    test_imgs.append(img)
    test_ground_truth.append(0)

test_imgs = np.array(test_imgs)
test_ground_truth = np.array(test_ground_truth)

total = len(test_imgs)#*len(reference_imgs)
print(total)
correct = 0
for i in range(len(test_imgs)):
    test_pairs = []
    for ref in reference_imgs:
        test_pairs.append((ref, test_imgs[i]))
    test_pairs = np.array(test_pairs)
    prediction = siamese_net.predict([test_pairs[:, 0], test_pairs[:, 1]])
    ones = 0
    zeros = 0
    gt = test_ground_truth[i]
    #print(prediction)
    for val in prediction:
        if val < 0.5 : zeros += 1
        else : ones += 1
    print(gt,zeros,ones)
    if gt == 0 :
      if zeros >= ones : correct += 1
    elif gt == 1 : 
      if ones >= zeros : correct += 1

result = correct / total
print("Accuracy : ")
print(result)
'''siamese_net.save("siamese_cnn_for_glaucoma_model.h5")
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
uploaded = drive.CreateFile({'title': 'siamese_cnn_for_glaucoma_model.h5'})
uploaded.SetContentFile('siamese_cnn_for_glaucoma_model.h5')
uploaded.Upload()'''


Instructions for updating:
Use tf.cast instead.
41
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 0 15
1 1 14
1 0 15
1 0 15
1 0 15
1 0 15
1 15 0
0 0 15
0 14 1
0 0 15
0 0 15
0 12 3
0 11 4
0 15 0
0 15 0
0 3 12
0 14 1
0 15 0
0 14 1
0 15 0
Accuracy : 
0.8780487804878049


'siamese_net.save("siamese_cnn_for_glaucoma_model.h5")\ngauth.credentials = GoogleCredentials.get_application_default()\ndrive = GoogleDrive(gauth)\nuploaded = drive.CreateFile({\'title\': \'siamese_cnn_for_glaucoma_model.h5\'})\nuploaded.SetContentFile(\'siamese_cnn_for_glaucoma_model.h5\')\nuploaded.Upload()'